# Thông tin
- Họ tên: Nguyễn Phúc Thuần
- MSSV: 20120380

# Bài làm

## 7.1
### (a)


In [147]:
p = 541
q = 1223
v = 159853

n = p * q
phi = (p-1)*(q-1)

s = pow(v, -1, phi)
print(f's = {s}, phi = {phi}, v = {v}, N = {n},')

s = 561517, phi = 659880, v = 159853, N = 661643,


Công khai: N = 661643, v = 159853

Private signing key: s = 561517

### (b)
Chữ kí số: $S = D^s \left(mod N\right)$

In [148]:
D = 630579
N = n
S = pow(D, s, N)
print(f'Signature S = {S}')

Signature S = 206484


## 7.2


In [149]:
C = lambda S,v: pow(S, v, N)
validate = lambda D, S, v: D == C(S,v)

(D_pure, S_pure) = (119812, 876453)
(D_quote, S_quote) = (161153, 870099)
(D_2quote, S_2quote) = (586036, 602754)
v = 87953
N = 1562501

print(validate(D_pure, S_pure, v))
print(validate(D_quote, S_quote, v))
print(validate(D_2quote, S_2quote, v))


False
True
True


## 7.6

Ta có: 

$v^{S_1}S_1^{S_2} \equiv v^{S_1}(g_iv_j)^{-S_1j^{-1}} \equiv v^{S_1}g^{-ij^{-1}S_1}v^{-S_1} \equiv g^{-ij^{-1}S_1} \equiv g^D \left(mod p\right)$ (đpcm)


## 7.8


In [150]:
(p, q, g) = (22531, 751, 4488)
s = 674
D = 244
e = 574

v = pow(g, s, p)
print(f'Public verification key v = {v}')

S1 = pow(g, e, p) % q
S2 = ((D + s*S1) * pow(e, -1, q)) % q

print(f'Signature (S1, S2) = {(S1, S2)}')

Public verification key v = 4940
Signature (S1, S2) = (444, 56)


## 7.9

In [151]:
(p, q, g) = (22531, 751, 4488)
v = 22476

def validate_DSA(sign, D):
    (S1, S2) = sign
    V1 = (D*pow(S2, -1, q)) % q
    V2 = S1*pow(S2, -1, q) % q
    
    print(f'V1 = {V1}, V2 = {V2}')
    
    return S1 == (pow(g, V1) * pow(v, V2) % p) % q

# (a)
(S1, S2) = (183, 260)
D = 329
print(validate_DSA((S1, S2), D))

# (b)
(S1, S2) = (211, 97)
D = 432
print(validate_DSA((S1, S2), D))

V1 = 293, V2 = 252
True
V1 = 709, V2 = 428
False


## 7.12


In [152]:
def inverse(x, p):
    if x % p == 0:
        return None
    
    return pow(x, p-2, p)
    
def ellyptic_addition(P, Q, E, p):
    __O__ = (0,0)
    if P == __O__:
        return Q
    
    if Q == __O__:
        return P
    
    x1, y1 = P
    x2, y2 = Q
    
    if x1 == x2 and y1 == -y2:
        return __O__
    
    if P != Q:
        denom = inverse(x2 - x1, p)
        if denom == None:
            return __O__
        
        __lambda__ = ((y2 - y1)*denom) % p
    else:
        A,_ = E
        denom = inverse(2*y1, p)
        if denom == None:
            return __O__
        
        __lambda__ = ((3*pow(x1, 2) + A) * denom) % p
        
    x3 = pow(__lambda__, 2) - x1 - x2
    y3 = __lambda__*(x1 - x3) - y1
    
    return (x3 % p, y3 % p)

def square_root(x, p):
    return [i for i in range(0,p) if pow(i, 2, p) == x]

def elliptic_points(a, b, p):
    E_points = [(0,0)]
    
    for x in range(0,p):
        yy = pow(x, 3) + a*x + b
        sqr = square_root(yy % p, p)
    
    if len(sqr) > 0:
        for y in sqr:
            E_points.append((x, y))
            
    return E_points

def double_and_add(P, n, a, b, p):
    
    Q = P
    R = (0,0)
    
    while n > 0:
        if n & 1:
            R = ellyptic_addition(R, Q, (a, b), p)
        Q = ellyptic_addition(Q, Q, (a, b), p)
        n = n >> 1
    
    return R

In [153]:
def verify_order_in_ELC(G, q, E, p):
    (A,B) = E
    qG = double_and_add(G, q, A, B, p)
    return qG == (0,0)
    

In [155]:
E = (231, 473)
p = 17389
q = 1321
G = (11259, 11278) 

if not verify_order_in_ELC(G,q, E, p):
    print(f'{G} is not a point of order {q} in E')
else:
    # V = sG
    s = 542
    A,B = E
    
    print('(a)')
    V = double_and_add(G, s, A, B, p)
    print(f'Public verification key V = {V}')
    
    d = 644
    e = 847
    
    eG = double_and_add(G, e, A, B, p)
    x,_ = eG
    s1 = x % q
    s2 = (d + s*s1)*pow(e, -1, q) % q
    print(f'Signature (s1, s2) = {(s1, s2)} mod {q}')
    
    print('(b)')
    V = (11017, 14637)
    (s1, s2) = (907, 296)
    d = 993
    v1 = d*pow(s2, -1, q) % q
    v2 = s1*pow(s2, -1, q) % q
    v1G = double_and_add(G, v1, A, B, p)
    v2V = double_and_add(V, v2, A, B, p)
    (x, _) = ellyptic_addition(v1G, v2V, (A,B), p)
    print(s1 == x % q)
    
    print('(c)')
    V = (14594, 308)
    #using brute force
    s = 0
    V_try = (0,0)
    while V_try != V:
        s = s + 1
        V_try = ellyptic_addition(V_try, G, (A, B), p)
        
    print(f'Private signing key of Umberto is: s = {s}')
    d = 516
    e = 365
    x,_ = double_and_add(G, e, A, B, p)
    s1 = x % q
    s2 = (d + s*s1)*pow(e, -1, q) % q
    print(f'Signature (s1, s2) = {(s1, s2)}')

(a)
Public verification key V = (8689, 1726)
Signature (s1, s2) = (491, 290) mod 1321
(b)
True
(c)
Private signing key of Umberto is: s = 1294
Signature (s1, s2) = (1281, 236)
